In [15]:
# ML imports
import pandas as pd #circular import?
import torch
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms
from PIL import Image
import timm
from sklearn.model_selection import train_test_split
from prodigyopt import Prodigy

#Logging imports
import csv
import os
import datetime
from pathlib import Path

#Nice to have imports
from pathlib import Path
from tqdm import tqdm
from mac_alerts import alerts

print(torch.__version__)

2.2.0


In [16]:
# For logging
# Define the path to the log file
log_file_path = os.path.join('logs', 'training_log.csv')

# Create the logs directory if it doesn't exist
if not os.path.exists('logs'):
    os.makedirs('logs')

# Open the log file in append mode
with open(log_file_path, 'a', newline='') as log_file:
    log_writer = csv.writer(log_file)
    # Write the header if the file is empty
    if os.path.getsize(log_file_path) ==   0:
        log_writer.writerow(['Timestamp', 'Epoch', 'Batch', 'Loss', 'Val Loss', 'Best Val Loss', 'Improved'])


In [17]:
# PARAMS
single_shot = False
learning_rate = 0.001 #Doesn't matter when using prodigy optimizer
epochs = 50 #Max_limit

In [18]:
# MODEL AND ARCHITECTURE
# Initialize the model
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=5) # Get model architecture
# Specify the path to your model directory
model_dir = 'models'
model_file = 'best_model.pt'
model_path = os.path.join(model_dir, model_file)

# Check if the model directory exists, if not, create it
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Check if the model file exists, if not, handle accordingly (e.g., download or initialize)
if not os.path.exists(model_path):
    print(f"Model file {model_path} does not exist. Using pretrained model.")
else:
    # Load the state dictionary from the saved model
    state_dict = torch.load(model_path)
    # Update the model's state dictionary with the loaded state dictionary
    model.load_state_dict(state_dict)

# If you're using a GPU, you might need to move the model to the GPU
if torch.cuda.is_available():
    model = model.cuda()


In [ ]:
# For logging
# Define the path to the log file
log_file_path = os.path.join('logs', 'training_log.csv')

# Create the logs directory if it doesn't exist
if not os.path.exists('logs'):
    os.makedirs('logs')

# Open the log file in append mode
with open(log_file_path, 'a', newline='') as log_file:
    log_writer = csv.writer(log_file)
    # Write the header if the file is empty
    if os.path.getsize(log_file_path) ==   0:
        log_writer.writerow(['Timestamp', 'Epoch', 'Batch', 'Loss', 'Val Loss', 'Best Val Loss', 'Improved'])


In [ ]:
# For logging
# Define the path to the log file
log_file_path = os.path.join('logs', 'training_log.csv')

# Create the logs directory if it doesn't exist
if not os.path.exists('logs'):
    os.makedirs('logs')

# Open the log file in append mode
with open(log_file_path, 'a', newline='') as log_file:
    log_writer = csv.writer(log_file)
    # Write the header if the file is empty
    if os.path.getsize(log_file_path) ==   0:
        log_writer.writerow(['Timestamp', 'Epoch', 'Batch', 'Loss', 'Val Loss', 'Best Val Loss', 'Improved'])


In [ ]:
# For logging
# Define the path to the log file
log_file_path = os.path.join('logs', 'training_log.csv')

# Create the logs directory if it doesn't exist
if not os.path.exists('logs'):
    os.makedirs('logs')

# Open the log file in append mode
with open(log_file_path, 'a', newline='') as log_file:
    log_writer = csv.writer(log_file)
    # Write the header if the file is empty
    if os.path.getsize(log_file_path) ==   0:
        log_writer.writerow(['Timestamp', 'Epoch', 'Batch', 'Loss', 'Val Loss', 'Best Val Loss', 'Improved'])


In [ ]:
# For logging
# Define the path to the log file
log_file_path = os.path.join('logs', 'training_log.csv')

# Create the logs directory if it doesn't exist
if not os.path.exists('logs'):
    os.makedirs('logs')

# Open the log file in append mode
with open(log_file_path, 'a', newline='') as log_file:
    log_writer = csv.writer(log_file)
    # Write the header if the file is empty
    if os.path.getsize(log_file_path) ==   0:
        log_writer.writerow(['Timestamp', 'Epoch', 'Batch', 'Loss', 'Val Loss', 'Best Val Loss', 'Improved'])


In [ ]:
# For logging
# Define the path to the log file
log_file_path = os.path.join('logs', 'training_log.csv')

# Create the logs directory if it doesn't exist
if not os.path.exists('logs'):
    os.makedirs('logs')

# Open the log file in append mode
with open(log_file_path, 'a', newline='') as log_file:
    log_writer = csv.writer(log_file)
    # Write the header if the file is empty
    if os.path.getsize(log_file_path) ==   0:
        log_writer.writerow(['Timestamp', 'Epoch', 'Batch', 'Loss', 'Val Loss', 'Best Val Loss', 'Improved'])


In [19]:
# WEIGHTED RANDOM SAMPLER

# score_counts = df['score'].value_counts()
# percentage = score_counts / len(df) * 100
# print(score_counts, percentage)

weights = torch.tensor([0.32096837,  0.17649356,  0.13705584,  0.04412339, 0.32135884])
sampler = WeightedRandomSampler(weights, len(weights))

'''
score
4.0    823
0.0    822
1.0    452
2.0    351
3.0    113
Name: count, dtype: int64 score
4.0    32.135884
0.0    32.096837
1.0    17.649356
2.0    13.705584
3.0     4.412339
Name: count, dtype: float64
'''

'\nscore\n4.0    823\n0.0    822\n1.0    452\n2.0    351\n3.0    113\nName: count, dtype: int64 score\n4.0    32.135884\n0.0    32.096837\n1.0    17.649356\n2.0    13.705584\n3.0     4.412339\nName: count, dtype: float64\n'

In [20]:
#DATA LABELS
#import pandas as pd
df = pd.read_csv('img_labels_ALL.csv')


class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        img_path = f'img/{img_name}'
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.df.iloc[idx, 2], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

# Define the transformations
# transform = transforms.Compose([
#     transforms.Resize((224,   224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485,   0.456,   0.406], std=[0.229,   0.224,   0.225]),
# ])

transform = timm.data.create_transform(
    **timm.data.resolve_data_config(model.pretrained_cfg))

# TEST SET
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
test_data = CustomDataset(test_df, transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# VALIDATION SET + TRAINING SET
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
# Create data loaders for training and validation sets
train_data = CustomDataset(train_df, transform)
val_data = CustomDataset(val_df, transform)

train_loader = DataLoader(train_data, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = Prodigy(model.parameters())


In [21]:
# SETUP"

# Define a function to evaluate the model on the validation set
def evaluate_on_validation_set(model, val_loader):
    model.eval()
    total_loss =  0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.long())
            total_loss += loss.item()
    return total_loss / len(val_loader)


# Get the current working directory
cwd = os.getcwd()

# Define paths for saving models relative to the current working directory
model_dir = os.path.join(cwd, "models")
best_model_path = os.path.join(model_dir, "best_model.pt")
last_model_path = os.path.join(model_dir, "last_model.pt")

# Initialize best validation loss
best_val_loss = float('inf')

In [22]:
#TRAINING LOOP
# Wrap both loops with tqdm for progress visualization
for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    for i, data in tqdm(enumerate(train_loader,   0), desc=f"Epoch {epoch +  1}", leave=False):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())  # Ensure labels are long integers
        loss.backward()
        optimizer.step()
        # Save weights per batch
        torch.save(model.state_dict(), last_model_path)
        print(f"Batch {i + 1}/{len(train_loader)}: Loss {loss.item():.4f}", end="\r")

        # Log the loss and other metrics
        with open(log_file_path, 'a', newline='') as log_file:
            log_writer = csv.writer(log_file)
            log_writer.writerow([datetime.datetime.now(), epoch +   1, i +   1, loss.item(), val_loss, best_val_loss, improved])


        if single_shot:
            break #after the first batch

    # Evaluate the model on the validation set
    val_loss = evaluate_on_validation_set(model, val_loader)  # Replace with your actual evaluation function
    improved = val_loss < best_val_loss

    # Check if the current model is better than the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)

    print(f"Epoch: {epoch + 1}/{epochs}, Val Loss: {val_loss:.4f}, Best Val Loss: {best_val_loss:.4f}, Improved: {improved}")
    
    # Log the loss and other metrics
    with open(log_file_path, 'a', newline='') as log_file:
        log_writer = csv.writer(log_file)
        log_writer.writerow([datetime.datetime.now(), epoch +   1, i +   1, loss.item(), val_loss, best_val_loss, improved])




# alerts.play_success()
# os.system('say "Træning færdig"'); print('\a\a\a')

  0%|          | 0/50 [00:17<?, ?it/s]


KeyboardInterrupt: 

## parallel evaluations of test set

In [ ]:
from sklearn.metrics import classification_report
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def evaluate_single_sample(i):
    # Get the sample from the dataset
    sample, true_label = test_data[i]
    
    # Pass the sample through the model
    with torch.no_grad():
        prediction = model(sample.unsqueeze(0))  # Unsqueeze to add batch dimension
    
    # Convert the prediction to a class label
    _, predicted_class = torch.max(prediction, dim=1)
    
    # Return the true label and the predicted class
    return true_label.item(), predicted_class.item()

# Ensure the model is in evaluation mode
model.eval()

# Shared lists to store true labels and predicted classes
true_labels_list = []
predicted_classes_list = []

# Create a thread pool executor
with ThreadPoolExecutor() as executor:
    # Evaluate all samples in parallel and collect results
    results = list(tqdm(executor.map(evaluate_single_sample, range(len(test_data))), total=len(test_data), desc='Evaluating'))

# Collect the true labels and predicted classes from the results
for true_label, predicted_class in results:
    true_labels_list.append(true_label)
    predicted_classes_list.append(predicted_class)

# Generate the classification report
report = classification_report(true_labels_list, predicted_classes_list)

# Print the classification report
print(report)

# Play a success sound
from mac_alerts import alerts
import os
alerts.play_success()
os.system('say "Evaluering færdig"') 
print('\a\a\a')

Evaluating:   0%|          | 0/513 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 513/513 [01:15<00:00,  6.83it/s]
/Users/alexandermittet/miniconda3/envs/BA/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/alexandermittet/miniconda3/envs/BA/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/alexandermittet/miniconda3/envs/BA/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

              precision    recall  f1-score   support

           0       0.32      0.98      0.49       165
           1       0.00      0.00      0.00        87
           2       0.00      0.00      0.00        84
           3       0.00      0.00      0.00        15
           4       0.15      0.01      0.02       162

    accuracy                           0.32       513
   macro avg       0.10      0.20      0.10       513
weighted avg       0.15      0.32      0.16       513


